In [24]:
import pandas as pd
import xarray as xr
from fastapi import FastAPI, HTTPException
from datetime import datetime, timedelta
from typing import Optional
import polars as pl
import numpy as np
#import uvloop
#import asyncio
#import polars as pl
#import zarr, dask
import time

In [2]:
st = time.time()
dz = xr.open_zarr(
    'sst_anomaly.zarr', chunks='auto', 
    group='anomaly', decode_times=True)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz

Exe time:  0.18165302276611328 sec


<xarray.Dataset>
Dimensions:      (date: 498, lat: 720, var: 3, lon: 1440)
Coordinates:
  * date         (date) datetime64[ns] 1982-01-01 1982-02-01 ... 2023-06-01
  * lat          (lat) float64 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon          (lon) float64 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * var          (var) object 'sst_anomaly' 'level' 'td'
Data variables:
    level        (var, date, lat, lon) float64 dask.array<chunksize=(1, 3, 180, 360), meta=np.ndarray>
    sst_anomaly  (var, date, lat, lon) float64 dask.array<chunksize=(1, 3, 180, 360), meta=np.ndarray>
    td           (var, date, lat, lon) float64 dask.array<chunksize=(1, 3, 180, 360), meta=np.ndarray>

In [27]:
def to_nearest_grid_point(lon: float, lat: float) -> tuple:
    grid_lon = round(lon * 4) / 4
    grid_lat = round(lat * 4) / 4
    return (grid_lon, grid_lat)

In [60]:
def read_mhw(lon0: float, lat0: float, lon1: Optional[float] = None, lat1: Optional[float] = None, start: Optional[str] = None, end: Optional[str] = None, append: Optional[str] = None):
    # Load your dataset
    #dz = xr.open_zarr('sst_anomaly.zarr', chunks='auto', group='anomaly', decode_times=True)
    global dz

    # Check date strings
    # start_date = mhw_params.start or dz.date.min().values
    # end_date = mhw_params.end or dz.date.max().values
    if start is None:
        start_date = pd.to_datetime('1982-01-01')
    else:
        try:
            start_date = pd.to_datetime(start)
            start_date = start_date.replace(day=1)  # set day to 1
        except ValueError:
            raise HTTPException(status_code=400, detail="Invalid start date format")

    if end is None:
        end_date = pd.to_datetime(datetime.today())
    else:
        try:
            end_date = pd.to_datetime(end)
            end_date = end_date.replace(day=1)  # set day to 1
        except ValueError:
            raise HTTPException(status_code=400, detail="Invalid end date format")

    # Convert longitude and latitude
    #lon0 = lon0 + 180
    #lon1 = lon1 + 180
    try:
        lon0, lat0 = to_nearest_grid_point(lon0, lat0)

        if lon1 is None or lat1 is None:
            # Only one point, no date range limitation
            data_subset = dz.sel(lon=slice(lon0+180, lon0+180.24), lat=slice(lat0, lat0+0.24), date=slice(start_date, end_date))
    
        else:
            # Bounding box, 1 year date range limitation
            lon1, lat1 = to_nearest_grid_point(lon1, lat1)
            if (end_date - start_date).days > 365:
                end_date = start_date + timedelta(days=365)
             
            data_subset = dz.sel(lon=slice(lon0+180, lon1+180), lat=slice(lat0, lat1), date=slice(start_date, end_date))

        if data_subset.nbytes == 0:
            raise HTTPException(status_code=400, detail="No data available for the given parameters.")

        if append is None:
            append = 'level'

        variables = [var.strip() for var in append.split(',')]
        df = data_subset.to_dataframe().reset_index()
        df['lon'] = df['lon'] - 180 #(df['lon'] - 1) / 4 - 180  # convert back to -180 to 180
        df = df[['lon', 'lat', 'date'] + variables].dropna(how='all', subset=variables)

        if df.empty:
            raise HTTPException(status_code=400, detail="No data available after removing rows with NA values.")

        return df #pl.from_pandas(df).to_dict()
    
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))


In [55]:
dt = read_mhw(lon0=120, lat0=20, lon1=130, lat1=30, start='2023-05-01', end='2023-06-30', append='level')
print(dt)

          lon     lat       date  level
40    120.125  20.125 2023-05-01    1.0
41    120.375  20.125 2023-05-01    1.0
42    120.625  20.125 2023-05-01    0.0
43    120.875  20.125 2023-05-01    0.0
44    121.125  20.125 2023-05-01    0.0
...       ...     ...        ...    ...
9555  128.875  29.875 2023-06-01    1.0
9556  129.125  29.875 2023-06-01    1.0
9557  129.375  29.875 2023-06-01    1.0
9558  129.625  29.875 2023-06-01    1.0
9559  129.875  29.875 2023-06-01    1.0

[3200 rows x 4 columns]


In [56]:
dt = read_mhw(lon0=115, lon1=125.5, lat0=19, lat1=29.5, start='2021-05-01', end='2023-06-30', append='level,sst_anomaly')
print(dt)

           lon     lat       date  level  sst_anomaly
0      115.125  19.125 2021-05-01    NaN     0.353762
1      115.375  19.125 2021-05-01    NaN     0.357706
2      115.625  19.125 2021-05-01    NaN     0.368999
3      115.875  19.125 2021-05-01    NaN     0.387293
4      116.125  19.125 2021-05-01    NaN     0.388624
...        ...     ...        ...    ...          ...
68749  124.375  29.375 2022-05-01    1.0          NaN
68750  124.625  29.375 2022-05-01    1.0          NaN
68751  124.875  29.375 2022-05-01    1.0          NaN
68752  125.125  29.375 2022-05-01    1.0          NaN
68753  125.375  29.375 2022-05-01    1.0          NaN

[45864 rows x 5 columns]


In [66]:
dt = read_mhw(lon0=135, lat0=15)
print(dt)

          lon     lat       date  level
1     135.125  15.125 1982-01-01    0.0
4     135.125  15.125 1982-02-01    0.0
7     135.125  15.125 1982-03-01    0.0
10    135.125  15.125 1982-04-01    0.0
13    135.125  15.125 1982-05-01    0.0
...       ...     ...        ...    ...
1480  135.125  15.125 2023-02-01    0.0
1483  135.125  15.125 2023-03-01    0.0
1486  135.125  15.125 2023-04-01    0.0
1489  135.125  15.125 2023-05-01    0.0
1492  135.125  15.125 2023-06-01    2.0

[498 rows x 4 columns]
